In [2]:
!pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 31.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LeakyReLU, Permute, Lambda, Multiply, Input, LayerNormalization, RepeatVector, Bidirectional, InputLayer, Conv1D, MaxPool1D, Conv2D, Conv3D, MaxPooling3D, BatchNormalization, MaxPool2D, GlobalMaxPool2D, GlobalAveragePooling2D, TimeDistributed, GRU, Dense, LSTM, Dropout, Activation, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard 
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adadelta, Nadam, Adagrad
from sklearn.model_selection import train_test_split, LeaveOneGroupOut, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit, TimeSeriesSplit, KFold, GroupKFold
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from keras.utils import to_categorical
from sklearn.preprocessing import RobustScaler
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
import matplotlib.pyplot as plt
import statistics
from sklearn import metrics
import time
import glob
import numpy as np
import pandas as pd
from keras import backend as K
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from scipy.signal import butter, filtfilt

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU:", tf.test.gpu_device_name())
else:
    print("TensorFlow is not using a GPU")

TensorFlow version: 2.9.2
Num GPUs Available:  0
TensorFlow is not using a GPU


2025-03-05 17:29:37.044858: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


In [ ]:
class_labels = ['Non-Func','Func']
duration = 60

In [ ]:
l2 = regularizers.l2(l=1e-3)
l1_l2 = regularizers.l1_l2(l1=0.01, l2=0.01)
momentum = 0.9

In [ ]:
def build_cnn2D(shape=(duration, 3, 1)):
    opt = Adam(learning_rate=1e-3, clipvalue=0.5)

    model = Sequential([
        Conv2D(16, (2,2), padding='same', input_shape=shape),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),
        
        Conv2D(32, (2,2), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),
        
        Flatten(),
        
        Dense(64, activation='relu', kernel_regularizer=l2
             ),
        Dropout(0.3),
        
        Dense(64, activation='relu', kernel_regularizer=l2
             ),
        Dropout(0.3),
        
        Dense(1, activation='sigmoid', kernel_regularizer=l2
             )
    ])
    
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'], 
                  #sample_weight_mode="temporal"
                 )
    
    return model

In [ ]:
# Confusion matrix
def cm(X, Y, model):
    
    #model = tf.keras.models.load_model(model_path, custom_objects={'leaky_relu': tf.nn.leaky_relu})
    y_pred = model.predict(X)
    #y_pred = y_pred.round()
    y_pred = np.round(y_pred).tolist()
    print(len(Y[Y==0]), len(Y[Y==1]), len(y_pred))
    if (len(Y[Y==0])/len(Y) != 0) and (len(Y[Y==0])/len(Y) != 1):
        cm = confusion_matrix(Y, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = class_labels)
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.show()
        
        #Classification report
        print('Classification Report')
        print(classification_report(Y, y_pred, target_names = class_labels))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import statistics
import matplotlib.pyplot as plt

# Define the custom metrics and model building functions
# Make sure to define functions like `build_improved_cnngru`, `f1_m`, `precision_m`, `recall_m`, `TP`, `TN`, `FP`, `FN`, `cm` as required.

def create_model(x_0, y_0, z_0):
    fullshape = (duration, 3)
    train_accuracies = np.array([])
    val_accuracies = np.array([])

    sf = LeaveOneGroupOut()

    for train_index, val_index in sf.split(x_0, y_0, z_0):

        x_train, x_val = x_0[train_index], x_0[val_index]
        y_train, y_val = y_0[train_index], y_0[val_index]
        z_train, z_val = z_0[train_index], z_0[val_index]

        # Debug unique groups in training and validation sets
        print("Z train unique:", np.unique(z_train))
        print("Z val unique:", np.unique(z_val))

        # Reshape for scaling
        x_train_reshaped = x_train.reshape(-1, x_train.shape[2])
        x_val_reshaped = x_val.reshape(-1, x_val.shape[2])

        scaler = RobustScaler()
        x_train_reshaped = scaler.fit_transform(x_train_reshaped)
        x_val_reshaped = scaler.transform(x_val_reshaped)

        # Reshape back to original shape
        x_train = x_train_reshaped.reshape(x_train.shape)
        x_val = x_val_reshaped.reshape(x_val.shape)

        print('Train: ', len(y_train[y_train==0]), len(y_train[y_train==1]))
        print('Val: ', len(y_val[y_val==0]), len(y_val[y_val==1]))
        #if np.unique(z_val) in [14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34]:
        
        # Apply SMOTE to the training data
        #smote = SMOTE(sampling_strategy='auto', random_state=42)
        #x_train_reshaped = x_train.reshape(x_train.shape[0], -1)  # Reshape to 2D for SMOTE
        #x_train_resampled, y_train_resampled = smote.fit_resample(x_train_reshaped, y_train)
        #x_train = x_train_resampled.reshape(-1, fullshape[0], fullshape[1])  # Reshape back to 3D
        #y_train = y_train_resampled

        #print('Train: ', len(y_train[y_train==0]), len(y_train[y_train==1]))
        #print('Val: ', len(y_val[y_val==0]), len(y_val[y_val==1]))

        # Compute class weights
        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
        print("class weights: ", class_weights)
        class_weights = dict(zip(np.unique(y_train), class_weights))

        #modelts = build_improved_cnngru()
        modelts = build_cnn2D()

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-9)
        monitor = EarlyStopping(monitor='val_binary_accuracy', patience=20, mode='auto', restore_best_weights=True)

        history = modelts.fit(x_train, y_train, validation_data=(x_val, y_val), verbose=2,
                              callbacks=[reduce_lr, monitor], epochs=100, batch_size=32, #class_weight=class_weights
                             )

        train_accuracy = modelts.evaluate(x_train, y_train, verbose=2)
        train_accuracies = np.append(train_accuracies, train_accuracy[1])

        val_accuracy = modelts.evaluate(x_val, y_val, verbose=2)
        val_accuracies = np.append(val_accuracies, val_accuracy[1])

        print("Z val: ", np.unique(z_val))
        print("Train Acc: ", train_accuracy)
        print("Val Acc:", val_accuracy)

        print('Confusion Matrix for Training')
        cm(x_train, y_train, modelts)
        print('Confusion Matrix for Validation')
        cm(x_val, y_val, modelts)

        plt.plot(history.history['binary_accuracy'], label='Training accuracy')
        plt.plot(history.history['val_loss'], label='Validation accuracy')
        plt.title('Training Acc vs. Validation Acc')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Training', 'Validation'], loc='upper left')
        plt.show()

        plt.plot(history.history['loss'], label='Training loss')
        plt.plot(history.history['val_loss'], label='Validation loss')
        plt.title('Training Loss vs. Validation Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Training', 'Validation'], loc='upper left')
        plt.show()

    print("Mean Training Accuracy:", np.mean(train_accuracies))
    print("Std Training Accuracy:", statistics.pstdev(np.asarray(train_accuracies)))

    print("Mean Validation Accuracy:", np.mean(val_accuracies))
    print("Std Validation Accuracy:", statistics.pstdev(np.asarray(val_accuracies)))